In [2]:
from __future__ import print_function
import numpy as np
import random
import sys

In [9]:
text = open("shakespeare_final.txt","rb").read().lower()

In [12]:
characters = sorted(list(set(text)))

In [13]:
print('corpus length:', len(text))
print('total chars:', len(characters))

corpus length: 204359
total chars: 67


In [14]:
char2indices = dict((c, i) for i,c in enumerate(characters))
indices2char = dict((i,c) for i,c in enumerate(characters))

In [15]:
maxlen = 40
step =3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 68107


In [18]:
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
        y[i, char2indices[next_chars[i]]] = 1

<ipython-input-18-5fe6b2bfe8b3>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
<ipython-input-18-5fe6b2bfe8b3>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(characters)), dtype=np.bool)


In [26]:
X[0], X[0].shape

(array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 (40, 67))

In [28]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

model.add(LSTM(128, input_shape=(maxlen, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               100352    
                                                                 
 dense_2 (Dense)             (None, 67)                8643      
                                                                 
 activation_2 (Activation)   (None, 67)                0         
                                                                 
Total params: 108,995
Trainable params: 108,995
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [30]:
for iteration in range(1,30):
    print('-'*40)
    print('interation', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text)-maxlen-1)
    
    for diversity in [0.2, 0.7, 1.2]:
        print('\n------ diversity:', diversity)
        generated =''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('------ Generating with seed: "' + sentence +'"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] =1.
                
            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds,diversity)
            pred_char = indices2char[next_index]
            
            generated += pred_char
            sentence = sentence[1:]+pred_char
            
            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print('\nOne combination completed \n')

----------------------------------------
interation 1
533/533 [==============================] - 18s 35ms/step - loss: 1.7643

------ diversity: 0.2


TypeError: can only concatenate str (not "bytes") to str